In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()

query = """
SELECT
    *,
    SNOWFLAKE.CORTEX.SENTIMENT(CUSTOMER_REVIEW) as SENTIMENT_SCORE
FROM 
    PARSED_REVIEWS
"""

df_reviews = session.sql(query).to_pandas()

df_reviews.head()

In [ ]:
import matplotlib.pyplot as plt

product_sentiment = df_reviews.groupby("PRODUCT")["SENTIMENT_SCORE"].mean().sort_values()
product_sentiment.plot(kind="barh", title="Average Sentiment by Product")
plt.xlabel("Sentiment Score")
plt.tight_layout()
plt.show()

In [ ]:
query = """
SELECT 
    SHIPPING_DATE,
    COUNT(*) AS SHIPMENT_COUNT
FROM 
    SHIPPING_LOGS
GROUP BY
    SHIPPING_DATE
ORDER BY 
    SHIPPING_DATE
"""

shipment_df = session.sql(query).to_pandas()

shipment_df.head()

In [ ]:
shipment_df.plot(x="SHIPPING_DATE", y="SHIPMENT_COUNT", kind="line", title="Shipments Per Day")
plt.ylabel("Number of Shipments")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
low_volume_days = shipment_df[shipment_df["SHIPMENT_COUNT"] < 5]
print("Low-volume shipping days:\n", low_volume_days)

In [ ]:
query = """
SELECT
    *
FROM 
    SHIPPING_LOGS
"""

df_shipping = session.sql(query).to_pandas()

df_shipping.head()

In [ ]:
carrier_counts = df_shipping.groupby("CARRIER")["ORDER_ID"].count()
carrier_counts.plot(kind="bar", title="Total Shipments by Carrier")
plt.ylabel("Number of Shipments")
plt.tight_layout()
plt.show()

In [ ]:
combined_df = df_reviews.merge(df_shipping, on="ORDER_ID", how="inner")
combined_df.head()

In [ ]:
# Save the merged DataFrame to a new table in Snowflake
session.write_pandas(combined_df, "COMBINED_REVIEWS_SHIPPING", auto_create_table=True, overwrite=True)

# Validate that the new table exists and looks correct
session.table("COMBINED_REVIEWS_SHIPPING").limit(5).show()